<a href="https://colab.research.google.com/github/arthur-grimaud/protrein_workshopB_eubic2024/blob/main/eubic_protrein_workshop_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Workshop session B: Phosphoproteomics data analysis

This workshop aims to explore various strategies for PTM dataset analysis, offering a comprehensive understanding of key processing and validation approaches for PTM studies. Participants will work with a dataset of synthetic phosphopeptides, covering all analysis steps from spectrum identification to the validation and scoring of phosphorylation sites. The results from the different approaches investigated will be compared to assess their respective limitations and advantages.

---


#Requirements

## Dataset

For this workshop you will be using a dataset published by [Ferries et al](https://pubs.acs.org/doi/10.1021/acs.jproteome.7b00337) of synthetic phosphopeptide analyzed on an Orbitrap Fusion. The raw file as well as the description of the experimental procedure are available here: https://ftp.pride.ebi.ac.uk/pride/data/archive/2019/09/PXD007058/.

*▶* For *this* workshop, two files have to be downloaded (TODO which ?) :

## Softwares

(TODO add installation links)



---



## 1-Spectra identification and scoring

The first tasks of this workshop will consist in performing spectra identification using various approach. Open-search will be conducted with MS-fragger via frag pipe. Closed search with the database search engine Comet

### Open-search with MS fragger

TODO


### Close-search with Comet

TODO



## 2-Workflow comparison

(TODO here we have to write thte code and visualization for comparing the results from the different searches)

In [1]:
print("Welcome to the Protrein workshop at EUBIC winterschool 2024")

Welcome to the Protrein workshop at EUBIC winterschool 2024
